In [52]:
import tkinter as tk
from tkinter import ttk
import sqlite3
from tkinter import filedialog
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tkinter import messagebox 
import sqlite3

tablo1_list=[]
tablo2_list=[]
tablo3_list=[]
tablo2_agirliklar=[]
tablo2_basliklar=[]
tablo2_sırano=[]
ogrenci_notlari_tablosu=[]
ogrenci_basarilari=[]

In [53]:


# Veritabanı bağlantısı oluştur
conn = sqlite3.connect("Proje2Database.db")
cursor = conn.cursor()

# Dersler tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS Dersler (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Adi TEXT NOT NULL,
    Kodu TEXT NOT NULL
)
""")


# Öğrenciler tablosu
cursor.execute("""
CREATE TABLE IF NOT EXISTS Ogrenciler (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    No INTEGER
)
""")
# Dönem içi çıktı tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS DersCikti (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    SiraNo INTEGER NOT NULL,
    Icerik TEXT NOT NULL,
    Ders_id INTEGER NOT NULL,
    FOREIGN KEY (Ders_id) REFERENCES Dersler (id)
)
""")

# Program çıktı tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS ProgramCikti (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    SiraNo INTEGER NOT NULL,
    Icerik TEXT NOT NULL,
    Ders_id INTEGER NOT NULL,
    FOREIGN KEY (Ders_id) REFERENCES Dersler (id)
)
""")

# Notlar tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS Notlar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    OgrenciId INTEGER NOT NULL,
    DersId INTEGER NOT NULL,
    Puan INTEGER NOT NULL,
    Tur TEXT NOT NULL,
    FOREIGN KEY (DersId) REFERENCES Dersler (id),
    FOREIGN KEY (OgrenciId) REFERENCES Ogrenciler (id)
)
""")

#  Puan turu tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS PuanTuru (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Tur TEXT NOT NULL,
    DersId INTEGER NOT NULL,
    Yuzdesi INTEGER Not NULL,
    FOREIGN KEY (DersId) REFERENCES Dersler (id)
    
)
""")

# Değişiklikleri kaydet ve bağlantıyı kapat
conn.commit()
conn.close()

print("Veritabanı ve tablolar başarıyla oluşturuldu.")


Veritabanı ve tablolar başarıyla oluşturuldu.


In [54]:
def dersleri_listeye_al():
    try:
        # Veritabanına bağlan
        conn = sqlite3.connect("Proje2Database.db")  # Veritabanı dosyasının adı
        cursor = conn.cursor()
        
        # Dersleri veritabanından çek
        cursor.execute("SELECT Id, Adi, Kodu FROM Dersler")
        dersler = cursor.fetchall()  # [(ders_adi1, ders_kodu1), (ders_adi2, ders_kodu2), ...]

        # Listeye dönüştürme
        ders_listesi = [f"{ders[0]} ({ders[1]}) ({ders[2]})" for ders in dersler]  # Örneğin: "Matematik (MAT101)"
        
        return ders_listesi

    except sqlite3.Error as e:
        print("Veritabanı hatası:", e)
        return []

    finally:
        # Bağlantıyı kapat
        if conn:
            conn.close()

In [55]:
pencere = tk.Tk()
pencere.title("Sekmeli Arayüz")
pencere.geometry("1200x800")  

# Notebook (Sekmeler) oluştur
sekme_defteri = ttk.Notebook(pencere)
sekme_defteri.pack(fill=tk.BOTH, expand=True)

# Yükleme Sekmesi
yukleme_sekmesi = ttk.Frame(sekme_defteri)
sekme_defteri.add(yukleme_sekmesi, text="Veri Tabanına Yükleme Ekranı")

# === Yükleme Sekmesi İçeriği ===
tk.Label(yukleme_sekmesi, text="Yükleme Ekranı", font=("Arial", 16, "bold")).pack(pady=10)

# === Ders Yükleme Alanı ===
ders_yukleme_frame = ttk.LabelFrame(yukleme_sekmesi, text="Ders Yükleme")
ders_yukleme_frame.pack(fill="x", padx=20, pady=10)

tk.Label(ders_yukleme_frame, text="Ders Adı:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_adi_entry = ttk.Entry(ders_yukleme_frame)
ders_adi_entry.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(ders_yukleme_frame, text="Ders Kodu:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
ders_kodu_entry = ttk.Entry(ders_yukleme_frame)
ders_kodu_entry.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

# === Crawler ile Ders ve Öğrenme çıktılarını webden alarak excele kaydetme ===

crawler_frame = ttk.LabelFrame(yukleme_sekmesi,text="Crawler ile çıktıları webden alma")
crawler_frame.pack(fill="x", padx=20, pady=10)

tk.Label(crawler_frame, text="Ders Linki:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_cikti_link_yaz = ttk.Entry(crawler_frame)
ders_cikti_link_yaz.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

# === Ders Çıktıları Yükleme ===
ders_ciktilari_frame = ttk.LabelFrame(yukleme_sekmesi, text="Ders Çıktıları Yükleme")
ders_ciktilari_frame.pack(fill="x", padx=20, pady=10)

tk.Label(ders_ciktilari_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox = ttk.Combobox(ders_ciktilari_frame, values=dersleri_listeye_al())
ders_sec_combobox.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(ders_ciktilari_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry = ttk.Entry(ders_ciktilari_frame)
dosya_sec_entry.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry.delete(0, tk.END)
    dosya_sec_entry.insert(0, dosya)

dosya_sec_butonu = ttk.Button(ders_ciktilari_frame, text="...", command=dosya_sec)
dosya_sec_butonu.grid(row=0, column=4, padx=10, pady=5)

# === Program Çıktıları Yükleme ===
program_ciktilari_yukleme_frame = ttk.LabelFrame(yukleme_sekmesi, text="Program Çıktıları Yükleme")
program_ciktilari_yukleme_frame.pack(fill="x", padx=20, pady=10)

tk.Label(program_ciktilari_yukleme_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox2 = ttk.Combobox(program_ciktilari_yukleme_frame, values=dersleri_listeye_al())
ders_sec_combobox2.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(program_ciktilari_yukleme_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry2 = ttk.Entry(program_ciktilari_yukleme_frame)
dosya_sec_entry2.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec2():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry2.delete(0, tk.END)
    dosya_sec_entry2.insert(0, dosya)

dosya_sec_butonu2 = ttk.Button(program_ciktilari_yukleme_frame, text="...", command=dosya_sec2)
dosya_sec_butonu2.grid(row=0, column=4, padx=10, pady=5)

# === Puan Türü Yükleme ===
puan_turu_yukle_frame = ttk.LabelFrame(yukleme_sekmesi, text="Puan Türü Yükleme")
puan_turu_yukle_frame.pack(fill="x", padx=20, pady=10)

tk.Label(puan_turu_yukle_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox3 = ttk.Combobox(puan_turu_yukle_frame, values=dersleri_listeye_al())
ders_sec_combobox3.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(puan_turu_yukle_frame, text="Tür Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
tur_sec_combobox = ttk.Combobox(puan_turu_yukle_frame, values=["Vize", "Final", "Odev1","Odev2","Quiz1","Quiz2"])
tur_sec_combobox.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

tk.Label(puan_turu_yukle_frame, text="Yüzde :").grid(row=0, column=4, padx=10, pady=5, sticky="w")
tur_sec_textbox = ttk.Entry(puan_turu_yukle_frame)
tur_sec_textbox.grid(row=0, column=5, padx=10, pady=5, sticky="ew")


# === Notları Yükleme ===
notlar_frame = ttk.LabelFrame(yukleme_sekmesi, text="Notları Yükleme")
notlar_frame.pack(fill="x", padx=20, pady=10)

tk.Label(notlar_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox4 = ttk.Combobox(notlar_frame, values=dersleri_listeye_al())
ders_sec_combobox4.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(notlar_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry4 = ttk.Entry(notlar_frame)
dosya_sec_entry4.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry4.delete(0, tk.END)
    dosya_sec_entry4.insert(0, dosya)

dosya_sec_butonu4 = ttk.Button(notlar_frame, text="...", command=dosya_sec)
dosya_sec_butonu4.grid(row=0, column=4, padx=10, pady=5)


# Çıktı Sekmesi
cikti_sekmesi = ttk.Frame(sekme_defteri)
sekme_defteri.add(cikti_sekmesi, text="ÇıktıLArı Oluşturma Ekranı")

# Çıktı Sekmesi İçeriği
tk.Label(cikti_sekmesi, text="Çıktı Ekranı", font=("Arial", 16)).pack(pady=20)

#Ders seçim Alanı

ders_secimi_frame =ttk.LabelFrame(cikti_sekmesi,text="Ders Seçimi")
ders_secimi_frame.pack(fill="x",padx=20,pady=10)

tk.Label(ders_secimi_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox7 = ttk.Combobox(ders_secimi_frame, values=dersleri_listeye_al())
ders_sec_combobox7.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

# === Tablo1 Yükleme ===
tablo1_frame = ttk.LabelFrame(cikti_sekmesi, text="Tablo1 Yükleme")
tablo1_frame.pack(fill="x", padx=20, pady=10)


                        
tk.Label(tablo1_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry6 = ttk.Entry(tablo1_frame)
dosya_sec_entry6.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry6.delete(0, tk.END)
    dosya_sec_entry6.insert(0, dosya)

dosya_sec_butonu6 = ttk.Button(tablo1_frame, text="...", command=dosya_sec)
dosya_sec_butonu6.grid(row=0, column=4, padx=10, pady=5)

# === tablo2 Yükleme ===
tablo2_frame = ttk.LabelFrame(cikti_sekmesi, text="tablo2 Yükleme")
tablo2_frame.pack(fill="x", padx=20, pady=10)



tk.Label(tablo2_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry5 = ttk.Entry(tablo2_frame)
dosya_sec_entry5.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry5.delete(0, tk.END)
    dosya_sec_entry5.insert(0, dosya)

dosya_sec_butonu4 = ttk.Button(tablo2_frame, text="...", command=dosya_sec)
dosya_sec_butonu4.grid(row=0, column=4, padx=10, pady=5)

#Ciktilari oluştur Frame

Ciktilari_olustur_frame = ttk.LabelFrame(cikti_sekmesi, text="Ciktilari Oluşturma")
Ciktilari_olustur_frame.pack(fill="x", padx=20, pady=10)





In [56]:
def ogrenme_ciktilari_al():
    url = ders_cikti_link_yaz.get()
    ders_cikti_link_yaz.delete(0, 'end')

    try:
        # HTTP isteği gönder
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # HTTP hatalarını kontrol et
    except requests.exceptions.RequestException as e:
        messagebox.showerror("Hata", f"Bağlantı hatası: {e}")
        return
    else:
        # HTML içeriğini işle
        soup = BeautifulSoup(response.text, 'html.parser')

    # Dersin Öğrenme Çıktıları Tablosu
    dersin_ogrenme_ciktilari = []
    dersin_ogrenme_ciktilari_table = soup.find('table', {'id': 'Content_Content_LearningOutcomes_gridLearningOutComes_DXMainTable'})
    if dersin_ogrenme_ciktilari_table:
        rows = dersin_ogrenme_ciktilari_table.find_all('tr')
        for row in rows:
            cols = row.find_all(['th', 'td'])
            cols = [col.text.strip() for col in cols]
            if cols:
                dersin_ogrenme_ciktilari.append(cols)
    else:
        messagebox.showwarning("Uyarı", "Dersin Öğrenme Çıktıları tablosu bulunamadı.")
        return

    # Program Öğrenme Çıktıları Tablosu
    program_ogrenme_ciktilari = []
    program_ogrenme_ciktilari_table = soup.find('table', {'id': 'Content_Content_DersinCiktilaraKatkisi_gridDersinCiktilaraEtkisi_DXMainTable'})
    if program_ogrenme_ciktilari_table:
        rows = program_ogrenme_ciktilari_table.find_all('tr')
        for row in rows:
            cols = row.find_all(['th', 'td'])
            cols = [col.text.strip() for col in cols]
            if cols:
                program_ogrenme_ciktilari.append(cols)
    else:
        messagebox.showwarning("Uyarı", "Program Yeterlilikleri tablosu bulunamadı.")
        return

    # Dersin Öğrenme Çıktıları verisini düzenle ve DataFrame'e dönüştür
    dersin_ogrenme_ciktilari_filtrele = [row[:2] for row in dersin_ogrenme_ciktilari[3:]]  # İlk 2 sütunu alıyoruz
    dersin_ogrenme_ciktilari_df = pd.DataFrame(dersin_ogrenme_ciktilari_filtrele, columns=["Sayı", "Öğrenme Çıktısı"])

    # Program Yeterlilikleri ile İlişkilendirilme verisini düzenle ve DataFrame'e dönüştür
    program_ogrenme_ciktilari_filtered = [row[:2] for row in program_ogrenme_ciktilari[10:]]  # İlk 2 sütunu alıyoruz
    program_ogrenme_ciktilari_df = pd.DataFrame(program_ogrenme_ciktilari_filtered, columns=["Sıra", "Program Öğrenme Çıktıları"])

    try:
        # Excel dosyalarına yazdır
        dersin_ogrenme_ciktilari_df.to_excel("Dersin_Öğrenme_Çıktıları.xlsx", index=False, sheet_name="Learning Outcomes")
        program_ogrenme_ciktilari_df.to_excel("Program_Öğrenme_Çıktıları.xlsx", index=False, sheet_name="Mapping Data")
        messagebox.showinfo("Başarılı", "Veriler başarıyla Excel dosyalarına kaydedildi.")
    except Exception as e:
        messagebox.showerror("Hata", f"Dosya kaydedilirken bir hata oluştu: {e}")


In [57]:
def program_cikti_ekle_buton_action():
    """
    Seçilen derse ait program çıktıları Excel dosyasından okuyup veritabanına ekler.
    """
    conn = None
    try:
        # --- 1. Ders ve Dosya Seçimi ---
        secilen_ders = ders_sec_combobox2.get().split(" ")[0]  # Ders ID'sini al
        print(f"Seçilen Ders: {secilen_ders}")

        if not secilen_ders:
            messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")
            return

        dosya_adi = dosya_sec_entry2.get()
        print(f"Seçilen Dosya: {dosya_adi}")

        if not dosya_adi:
            messagebox.showwarning("Uyarı", "Herhangi bir dosya seçilmedi.")
            return

        # --- 2. Veritabanı Bağlantısı ---
        conn = sqlite3.connect("Proje2Database.db")
        cursor = conn.cursor()
        print("Veritabanına bağlanıldı.")

        # Ders ID kontrolü
        cursor.execute("SELECT id FROM Dersler WHERE ID = ?", (secilen_ders,))
        ders_id_tuple = cursor.fetchone()
        print(f"Ders ID Tuple: {ders_id_tuple}")

        if not ders_id_tuple:
            messagebox.showerror("Hata", f"'{secilen_ders}' dersi veritabanında bulunamadı.")
            return

        ders_id = ders_id_tuple[0]
        print(f"Ders ID: {ders_id}")

        # --- 3. Excel Dosyasını Oku ---
        try:
            df = pd.read_excel(dosya_adi)
            print("Excel dosyası başarıyla okundu.")
        except Exception as e:
            messagebox.showerror("Hata", f"Excel dosyası okunamadı: {e}")
            return

        # Sıra ve İçerik sütunlarının alınıp listeye dönüştürülmesi
        if "Sıra" not in df.columns or "Program Öğrenme Çıktıları" not in df.columns:
            messagebox.showerror("Hata", "Excel dosyası beklenen formatta değil. 'Sıra' ve 'Öğrenme Çıktıları' sütunlarını kontrol edin.")
            return

        ciktilar = df[["Sıra", "Program Öğrenme Çıktıları"]].values.tolist()
        print(f"Çıktılar Listesi: {ciktilar}")

        # --- 4. Veritabanına Verileri Ekle ---
        for sira_no, icerik in ciktilar:
            cursor.execute(
                "INSERT INTO ProgramCikti (Ders_id, SiraNo, Icerik) VALUES (?, ?, ?)",
                (ders_id, sira_no, icerik),
            )
            print(f"Eklendi: Ders ID: {ders_id}, Sıra No: {sira_no}, İçerik: {icerik}")

        conn.commit()
        print("Değişiklikler kaydedildi.")
        messagebox.showinfo("Başarılı", "Program çıktıları başarıyla veritabanına eklendi.")

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")
    except sqlite3.Error as e:
        messagebox.showerror("Hata", f"Veritabanı hatası: {e}")
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")
    finally:
        if conn:
            conn.close()
            print("Veritabanı bağlantısı kapatıldı.")


In [58]:
def notlari_yukle_buton_action():
    # Dosya yolunu al
    dosya_yolu = dosya_sec_entry4.get()

    if not dosya_yolu:
        messagebox.showerror("Hata", "Lütfen bir dosya seçin.")
        return

    try:
        # Excel dosyasını oku
        df = pd.read_excel(dosya_yolu)
        global ogrenci_notlari_tablosu
        ogrenci_notlari_tablosu=df.values.tolist()
        # Başlıkları ve verileri yazdır
        print("Yüklenen Veriler (Tablo Formatında):")
        print(ogrenci_notlari_tablosu)

        # Başlıkların uzunluklarını bul
        column_widths = [max(len(str(item)) for item in df[column]) for column in df.columns]

        # Başlıkları yazdır (Genişliklere göre hizalanmış)
        header_row = " | ".join([f"{col:<{width}}" for col, width in zip(df.columns, column_widths)])
        print(header_row)
        print("-" * len(header_row))  # Başlıkların altında çizgi

        # Verileri yazdır
        for row in df.values:
            print(" | ".join([f"{str(item):<{column_widths[i]}}" for i, item in enumerate(row)]))

        # Verilerin başarıyla yüklendiğini kullanıcıya bildir
        messagebox.showinfo("Başarılı", "Notlar başarıyla yüklendi!")

    except Exception as e:
        # Hata durumunda mesaj göster
        messagebox.showerror("Hata", f"Dosya okunurken bir hata oluştu: {e}")


In [59]:
ders_matrisi={}
def tablo1_yukle_buton_action():
    global ders_matrisi  # Sözlüğe erişmek için global tanımlandı

    # Combobox'tan seçilen dersin adını al
    secilen_ders = ders_sec_combobox7.get()

    if not secilen_ders:  # Ders seçimi kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")  # Uyarı mesajı
        return

    # Dosya yolunu dosya_sec_entry6 widget'ından al
    dosya_adi = dosya_sec_entry6.get()

    if not dosya_adi:  # Dosya yolu kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir dosya seçin.")  # Uyarı mesajı
        return

    try:
        # Excel dosyasını oku
        df = pd.read_excel(dosya_adi, sheet_name=0, engine="openpyxl")

        # Gerekli sütunları seç (ilk 10 satır ve ders çıktıları ile ilişki değerlerini dahil et)
        tablo_verisi = df.iloc[1:, 1:]

        # Matrise dönüştür (liste içinde liste)
        matris = tablo_verisi.values.tolist()
        global tablo1_list
        tablo1_list=matris
        # Matrisi sözlüğe ekle
        ders_matrisi[secilen_ders] = matris

        messagebox.showinfo("Başarılı", f"{secilen_ders} dersi için tablo başarıyla okundu!")  # Başarı mesajı
        print(f"{secilen_ders} dersi için tablo başarıyla okundu")

        # Matris çıktısını yazdır (isteğe bağlı)
        print(f"{secilen_ders} İçeriği (Matris):")
        for satir in matris:
            print(satir)

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")  # Hata mesajı
    except ValueError as e:
        messagebox.showerror("Hata", f"Dosya okuma hatası: {e}")  # Hata mesajı
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")  # Genel hata mesajı


def tablo1_matrisi_goruntule():
    # Eğer ders_matrisi sözlüğü boşsa, uyarı mesajı göster
    if not ders_matrisi:
        messagebox.showwarning("Uyarı", " Lütfen bir des seçin!")  # Ders matrisi yüklenmediyse uyarı
        return

    sozluk_icerigi = ""
    
    # Her ders ve onun matrisi üzerinde döngü
    for ders, matris in ders_matrisi.items():
        # Eğer dersin matrisi boşsa, uyarı mesajı göster
        if not matris:
            messagebox.showwarning("Uyarı", f"{ders} dersinin matrisi boş.")  # Dersin matrisi boşsa uyarı
            continue
        
        sozluk_icerigi += f"Ders: {ders}\nMatris:\n"
        
        # Matrisi satırlara ayırarak, her satırı ekleyin
        for satir in matris:
            sozluk_icerigi += f"{satir}\n"
        
        sozluk_icerigi += "-" * 30 + "\n"  # Her dersin arasına ayırıcı çizgi ekle
    
    # Eğer sozluk_icerigi boşsa, yani hiç ders matrisi gösterilmeyecekse, uyarı mesajı göster
    if not sozluk_icerigi:
        messagebox.showwarning("Uyarı", "Gösterilecek ders matrisi bulunamadı.")  # Matrisi görüntülenebilir ders yoksa uyarı
        return

    # Tüm içerik messagebox içinde gösterilecek
    messagebox.showinfo("Ders Matrisi Sözlüğü", sozluk_icerigi)


buton = ttk.Button(tablo1_frame, text="Sözlüğü Görüntüle", command=tablo1_matrisi_goruntule)
buton.grid(row=0, column=6, padx=10, pady=5)

In [60]:
def ders_cikti_ekle_buton_action():
    """
    Seçilen derse ait ders çıktılarını Excel dosyasından okuyup veritabanına ekler.
    """
    conn = None
    try:
        # --- 1. Ders ve Dosya Seçimi ---
        secilen_ders = ders_sec_combobox.get()
        secilen_ders = secilen_ders.split(" ") [0]
        print(f"Seçilen Ders: {secilen_ders}")

        if not secilen_ders:
            messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")
            return

        dosya_adi = dosya_sec_entry.get()
        print(f"Seçilen Dosya: {dosya_adi}")

        if not dosya_adi:
            messagebox.showwarning("Uyarı", "Herhangi bir dosya seçilmedi.")
            return

        # --- 2. Ders ID'sini Bul ---
        conn = sqlite3.connect("Proje2Database.db")
        cursor = conn.cursor()
        print("Veritabanına bağlanıldı.")

        cursor.execute("SELECT id FROM Dersler WHERE ID = ?", (secilen_ders,))
        ders_id_tuple = cursor.fetchone()
        print(f"Ders ID Tuple: {ders_id_tuple}")

        if not ders_id_tuple:
            messagebox.showerror("Hata", f"'{secilen_ders}' dersi veritabanında bulunamadı.")
            return

        ders_id = ders_id_tuple[0]
        print(f"Ders ID: {ders_id}")

        # --- 3. Excel'den Verileri Oku ---
        df = pd.read_excel(dosya_adi)
        print("Excel dosyası okundu.")

        # Sütun başlıklarını atla ve sadece Sıra No ve İçerik sütunlarını al
        ciktilar = df.iloc[0:].values.tolist()
        print(f"Çıktılar Listesi: {ciktilar}")

        # --- 4. Veritabanına Ekle ---
        for sira_no, icerik in ciktilar:
            cursor.execute(
                "INSERT INTO DersCikti (Ders_id, SiraNo, Icerik) VALUES (?, ?, ?)",
                (ders_id, sira_no, icerik),
            )
            print(f"Eklendi: Ders ID: {ders_id}, Sıra No: {sira_no}, İçerik: {icerik}")

        conn.commit()
        print("Değişiklikler kaydedildi (commit).")
        messagebox.showinfo("Başarılı", "Ders çıktıları başarıyla veritabanına eklendi.")

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")
    except pd.errors.ParserError:
        messagebox.showerror("Hata", "Dosya okuma hatası. Geçerli bir Excel dosyası seçtiğinizden emin olun.")
    except sqlite3.Error as e:
        messagebox.showerror("Hata", f"Veritabanı hatası: {e}")
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")
    finally:
        if conn:
            conn.close()
            print("Veritabanı bağlantısı kapatıldı.")

In [61]:
def puan_turu_ekle_buton_action():
    try:
        ders_id = ders_sec_combobox3.get().split(" ")[0]  # Ders seçimini al
        tur = tur_sec_combobox.get()  # Tür seçimini al
        yuzdesi = tur_sec_textbox.get()  # Yüzde girişini al

        if not ders_id or not tur or not yuzdesi:
            messagebox.showerror("Hata", "Tüm alanları doldurmanız gerekiyor!")
            return

        if not yuzdesi.isdigit():
            messagebox.showerror("Hata", "Yüzde yalnızca sayısal bir değer olmalıdır!")
            return

        # Veritabanına ekleme
        conn = sqlite3.connect("Proje2Database.db")
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO PuanTuru (Tur, DersId, Yuzdesi) VALUES (?, ?, ?)",
            (tur, ders_id, int(yuzdesi))
        )
        conn.commit()
        conn.close()

        # Başarı mesajı
        messagebox.showinfo("Başarılı", "Puan türü başarıyla eklendi!")

        # Alanları temizle
        ders_sec_combobox3.set("")
        tur_sec_combobox.set("")
        tur_sec_textbox.delete(0, tk.END)

    except Exception as e:
        messagebox.showerror("Hata", f"Bir hata oluştu: {e}")


In [62]:
def tablo2_yukle_buton_action():
    # Dosya yolunu dosya_sec_entry5 widget'ından al
    dosya_adi = dosya_sec_entry5.get()
    secilen_ders = ders_sec_combobox7.get()  # Seçilen ders

    if not dosya_adi:  # Dosya yolu kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir dosya seçin.")  # Uyarı mesajı
        return

    if not secilen_ders:  # Ders seçimi kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")  # Uyarı mesajı
        return

    try:
        # Excel dosyasını oku
        df = pd.read_excel(dosya_adi, engine="openpyxl")

        # Tüm veriyi oku (ilk satır başlıkları da dahil)
        tablo_verisi = df  # Verinin tamamını oku

        # Matrise dönüştür (liste içinde liste)
        matris = tablo_verisi.values.tolist()



        global tablo2_basliklar
        tablo2_basliklar=matris[2][1:]
             

        global tablo2_sırano
        tablo2_sırano=[satir[1] for satir in matris[3:]]
        

        global tablo2_agirliklar
        tablo2_agirliklar=matris[1][2:len(matris[0])-1]
        

        global tablo2_list
        tablo2_list=[satir[2:] for satir in matris[3:]]
        print("KONTROL")
        print(tablo2_list)
        print("KONTROL")


        # Seçilen dersle ilişkilendir
        ders_matrisi[secilen_ders] = matris

        # Başarı mesajı
        messagebox.showinfo("Başarılı", f"{secilen_ders} için Tablo2 başarıyla okundu!")
        print(f"{secilen_ders} için Tablo2 başarıyla okundu")

        # Matris çıktısını yazdır (isteğe bağlı)
        print(f"{secilen_ders} Tablo2 İçeriği (Matris):")
        for satir in matris:
            print(satir)

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")  # Hata mesajı
    except ValueError as e:
        messagebox.showerror("Hata", f"Dosya okuma hatası: {e}")  # Hata mesajı
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")  # Genel hata mesajı

def tablo2_matrisi_goruntule():
    secilen_ders = ders_sec_combobox7.get()  # Seçilen dersi al

    # Ders seçimi kontrolü
    if not secilen_ders:
        messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")  # Ders seçimi yapılmadıysa uyarı
        return

    # Seçilen dersin matrisi kontrolü
    if secilen_ders not in ders_matrisi:  
        messagebox.showwarning("Uyarı", f"{secilen_ders} için bir tablo yüklenmedi!")  # Tablo yüklenmemişse uyarı
        return

    # Seçilen dersin matrisi alınır
    matris = ders_matrisi[secilen_ders]

    # Dersin matrisi boşsa uyarı göster
    if not matris:
        messagebox.showwarning("Uyarı", f"{secilen_ders} dersinin matrisi boş.")  # Dersin matrisi boşsa uyarı
        return

    # Matrisi içeriklerini göster
    matris_metni = "\n".join([", ".join(map(str, satir)) for satir in matris])  # Matrisi stringe çevir
    messagebox.showinfo(f"{secilen_ders} Matrisi", f"{secilen_ders} için matris:\n\n{matris_metni}")


# Tablo2 Matris Görüntüleme Butonu
tablo2_goruntule_butonu = ttk.Button(tablo2_frame, text="Tablo2 Matrisi Göster", command=tablo2_matrisi_goruntule)
tablo2_goruntule_butonu.grid(row=0, column=6, padx=10, pady=5)


In [63]:
def ekle_butonu_action():
    # Ders adı ve ders kodunu al
    ders_adi = ders_adi_entry.get()
    ders_kodu = ders_kodu_entry.get()
    
    # Eğer alanlar boşsa uyarı ver
    if not ders_adi or not ders_kodu:
        tk.messagebox.showwarning("Eksik Bilgi", "Lütfen tüm alanları doldurunuz.")
        return
    
    try:
        # Veritabanına bağlan
        conn = sqlite3.connect("Proje2Database.db")
        cursor = conn.cursor()
        
        # Girilen bilgileri veritabanına ekle
        cursor.execute("INSERT INTO Dersler (Adi, Kodu) VALUES (?, ?)", (ders_adi, ders_kodu))
        
        # Değişiklikleri kaydet ve bağlantıyı kapat
        conn.commit()
        conn.close()
        
        # Başarı mesajı
        tk.messagebox.showinfo("Başarılı", "Ders başarıyla eklendi.")
        
        # Alanları temizle
        ders_adi_entry.delete(0, tk.END)
        ders_kodu_entry.delete(0, tk.END)
        
    except Exception as e:
        tk.messagebox.showerror("Hata", f"Bir hata oluştu: {e}")
    
    ders_sec_combobox['values'] = dersleri_listeye_al()
    ders_sec_combobox2['values'] = dersleri_listeye_al()
    ders_sec_combobox3['values'] = dersleri_listeye_al()
    ders_sec_combobox4['values'] = dersleri_listeye_al()
    ders_sec_combobox7['values'] = dersleri_listeye_al()
    

In [64]:
def tablo3_olustur_butonu_action():
    global tablo2_agirliklar,tablo2_list
    print(tablo2_agirliklar)

    for i in range(len(tablo2_agirliklar)):
        tablo2_agirliklar[i]=int(tablo2_agirliklar[i])/100
    print(tablo2_agirliklar)

    print(tablo2_list)
    global tablo3_list
    for row in tablo2_list:
        multiplication = [tablo2_agirliklar[i] * row[i] for i in range(len(tablo2_agirliklar))]
        tablo3_list.append(multiplication)
    for row in tablo3_list:
        row.append(sum(row))
    print (tablo3_list)

    for i in range(len(tablo3_list)):
        tablo3_list[i].insert(0, tablo2_sırano[i])
    #tablo3-4-5 oluşturulacak
    df = pd.DataFrame(tablo3_list, columns=tablo2_basliklar)

   

    # Excel dosyasına yazdırıyoruz
    df.to_excel('tablo3.xlsx', sheet_name='Tablo3', index=False)

      



In [65]:
def tablo4_olustur_butonu_action():
    global tablo3_list, ogrenci_notlari_tablosu, tablo2_basliklar, ogrenci_basarilari

    # DataFrame'lere dönüştürme
    tablo3_df = pd.DataFrame(tablo3_list, columns=['Ders Çıktı'] + tablo2_basliklar[1:len(tablo2_basliklar)-1] + ['TOPLAM'])
    ogrenci_df = pd.DataFrame(ogrenci_notlari_tablosu, columns=['Ogrenci_No'] + tablo2_basliklar[1:len(tablo2_basliklar)-1])

    # Öğrenci başarılarını saklamak için listeyi temizle
    ogrenci_basarilari.clear()

    def hesapla_ders_notlari(ogrenci_notlari, ogrenci_no):
        sonuc_data = []
        ogrenci_basarisi = [ogrenci_no]

        # TABLO 4 ve öğrenci numarası başlıkları
        sonuc_data.append(['TABLO 4', ogrenci_no] + [''] * (len(tablo2_basliklar) + 2))

        # Sütun başlıkları ekle
        sonuc_data.append(['Ders Çıktı'] + tablo2_basliklar[1:len(tablo2_basliklar)-1] + ['TOPLAM', 'MAX', '% Başarı'])

        for i, row in tablo3_df.iterrows():
            ders_no = i + 1
            toplam_puan = 0
            max_puan = 0

            # Dinamik hesaplamalar
            ders_sonucu = [ders_no]
            for baslik in tablo2_basliklar[1:len(tablo2_basliklar)-1]:
                if baslik in ogrenci_notlari:
                    baslik_puan = ogrenci_notlari[baslik] * row[baslik]
                    toplam_puan += baslik_puan
                    max_puan += row[baslik] * 100
                    ders_sonucu.append(baslik_puan)
                else:
                    ders_sonucu.append(0)

            basari_orani = round((toplam_puan / max_puan) * 100, 2) if max_puan > 0 else 0

            # Başarı oranını listeye ekle
            ogrenci_basarisi.append(basari_orani)

            # Ders satırını tamamla
            ders_sonucu += [round(toplam_puan, 2), max_puan, basari_orani]
            sonuc_data.append(ders_sonucu)

        # Öğrenci başarısını genel listeye ekle
        ogrenci_basarilari.append(ogrenci_basarisi)
        return sonuc_data

    sonuc_data = []
    for i, ogrenci in ogrenci_df.iterrows():
        ogrenci_no = ogrenci['Ogrenci_No']
        ogrenci_sonuc = hesapla_ders_notlari(ogrenci, ogrenci_no)
        sonuc_data.extend(ogrenci_sonuc)

    # Sütun başlıkları
    sonuc_columns = ['Ders Çıktı'] + tablo2_basliklar[1:len(tablo2_basliklar)-1] + ['TOPLAM', 'MAX', '% Başarı']
    sonuc_columns = [''] + sonuc_columns

    try:
        sonuc_df = pd.DataFrame(sonuc_data)
        sonuc_df.to_excel('Tablo4.xlsx', sheet_name='Tüm Öğrenciler', index=False, header=False)
        print("Sonuçlar 'Tablo4.xlsx' dosyasına yazıldı.")

        # Başarıları ayrı bir dosyaya yaz
        basari_df = pd.DataFrame(ogrenci_basarilari)
        basari_df.to_excel('OgrenciBasarilari.xlsx', sheet_name='Başarılar', index=False, header=False)
        print("Başarılar 'OgrenciBasarilari.xlsx' dosyasına yazıldı.")

    except ValueError as e:
        print("Hata:", e)

In [66]:
def tablo5_olustur_butonu_action():
    global ogrenci_basarilari, tablo1_list

    sonuc_data = []

    for ogrenci in ogrenci_basarilari:
        ogrenci_no = ogrenci[0]
        sonuc_data.append(['TABLO5', ogrenci_no])
        sonuc_data.append(['Prg Çıktı'] + ogrenci[1:] + ['Ortalama'])

        for i, satir in enumerate(tablo1_list[1:], start=1):
            program_cikti = satir[0]
            carpanlar = satir[1:-1]  # Son elemanı da al

            # Başarıları ve çarpanları dinamik olarak eşleştir
            basarilar = ogrenci[1:]
            

            sonuc = [round(b * c, 2) for b, c in zip(basarilar, carpanlar)]
            ortalama = round(sum(sonuc) / len([c for c in carpanlar if c > 0]), 2) if carpanlar else 0

            sonuc_data.append([program_cikti] + sonuc + [ortalama])

        sonuc_data.append([])

    try:
        sonuc_df = pd.DataFrame(sonuc_data)
        sonuc_df.to_excel('Tablo5.xlsx', sheet_name='Tüm Öğrenciler', index=False, header=False)
        print("Sonuçlar 'Tablo5.xlsx' dosyasına yazıldı.")
    except ValueError as e:
        print("Hata:", e)

In [67]:
ekle_butonu = ttk.Button(ders_yukleme_frame, text="Dersi Yükle",command=ekle_butonu_action)
ekle_butonu.grid(row=0, column=4, padx=10, pady=5)

ogrenme_ciktilari_al_butonu = ttk.Button(crawler_frame, text="Öğrenme Çıktılarını Al", command=ogrenme_ciktilari_al)
ogrenme_ciktilari_al_butonu.grid(row=0, column=2, padx=10, pady=5)

ders_cikti_ekle_butonu = ttk.Button(ders_ciktilari_frame, text="Ders Çıktılarını Yükle", command=ders_cikti_ekle_buton_action)
ders_cikti_ekle_butonu.grid(row=0, column=5, padx=10, pady=5)

program_cikti_ekle_butonu = ttk.Button(program_ciktilari_yukleme_frame, text="Program Çıktılarını Yükle", command=program_cikti_ekle_buton_action)
program_cikti_ekle_butonu.grid(row=0, column=5, padx=10, pady=5)

puan_turu_ekle_butonu = ttk.Button(puan_turu_yukle_frame, text=" Yükle",command=puan_turu_ekle_buton_action)
puan_turu_ekle_butonu.grid(row=0, column=6, padx=10, pady=5)

notlari_yukle_butonu = ttk.Button(notlar_frame, text="Notları Yükle",command=notlari_yukle_buton_action)
notlari_yukle_butonu.grid(row=0, column=5, padx=10, pady=5)

tablo1_yukle_butonu = ttk.Button(tablo1_frame, text="Tablo1 Yükle", command=tablo1_yukle_buton_action)
tablo1_yukle_butonu.grid(row=0, column=5, padx=10, pady=5)

tablo2_yukle_butonu = ttk.Button(tablo2_frame, text="tablo2 Yükle", command=tablo2_yukle_buton_action)
tablo2_yukle_butonu.grid(row=0, column=5, padx=10, pady=5)

tablo3_olustur_butonu = ttk.Button(Ciktilari_olustur_frame, text="Tablo3 Oluştur", command=tablo3_olustur_butonu_action)
tablo3_olustur_butonu.grid(row=0, column=1, padx=10, pady=5)

tablo4_olustur_butonu = ttk.Button(Ciktilari_olustur_frame, text="Tablo4 Oluştur", command=tablo4_olustur_butonu_action)
tablo4_olustur_butonu.grid(row=0, column=2, padx=10, pady=5)

tablo5_olustur_butonu = ttk.Button(Ciktilari_olustur_frame, text="Tablo5 Oluştur", command=tablo5_olustur_butonu_action)
tablo5_olustur_butonu.grid(row=0, column=3, padx=10, pady=5)



In [ ]:
# Test kodu
def test_fonksiyonlar():
    try:
        dersleri_listeye_al()
        print("dersleri_listeye_al: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"dersleri_listeye_al: Hata oluştu - {e}")

    try:
        ogrenme_ciktilari_al()
        print("ogrenme_ciktilari_al: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"ogrenme_ciktilari_al: Hata oluştu - {e}")

    try:
        program_cikti_ekle_buton_action()
        print("program_cikti_ekle_buton_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"program_cikti_ekle_buton_action: Hata oluştu - {e}")

    try:
        notlari_yukle_buton_action()
        print("notlari_yukle_buton_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"notlari_yukle_buton_action: Hata oluştu - {e}")

    try:
        tablo1_yukle_buton_action()
        print("tablo1_yukle_buton_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"tablo1_yukle_buton_action: Hata oluştu - {e}")

    try:
        tablo1_matrisi_goruntule()
        print("tablo1_matrisi_goruntule: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"tablo1_matrisi_goruntule: Hata oluştu - {e}")

    try:
        ders_cikti_ekle_buton_action()
        print("ders_cikti_ekle_buton_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"ders_cikti_ekle_buton_action: Hata oluştu - {e}")

    try:
        puan_turu_ekle_buton_action()
        print("puan_turu_ekle_buton_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"puan_turu_ekle_buton_action: Hata oluştu - {e}")

    try:
        tablo2_yukle_buton_action()
        print("tablo2_yukle_buton_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"tablo2_yukle_buton_action: Hata oluştu - {e}")

    try:
        tablo2_matrisi_goruntule()
        print("tablo2_matrisi_goruntule: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"tablo2_matrisi_goruntule: Hata oluştu - {e}")

    try:
        ekle_butonu_action()
        print("ekle_butonu_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"ekle_butonu_action: Hata oluştu - {e}")

    try:
        tablo3_olustur_butonu_action()
        print("tablo3_olustur_butonu_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"tablo3_olustur_butonu_action: Hata oluştu - {e}")

    try:
        tablo4_olustur_butonu_action()
        print("tablo4_olustur_butonu_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"tablo4_olustur_butonu_action: Hata oluştu - {e}")

    try:
        tablo5_olustur_butonu_action()
        print("tablo5_olustur_butonu_action: Doğru şekilde çalıştı.")
    except Exception as e:
        print(f"tablo5_olustur_butonu_action: Hata oluştu - {e}")


In [68]:

# Pencereyi çalıştır
pencere.mainloop()

Yüklenen Veriler (Tablo Formatında):
[[230501034, 2, 51, 73, 27, 53], [230501035, 7, 30, 98, 80, 37], [230501036, 77, 72, 7, 92, 49], [230501037, 66, 53, 89, 53, 87], [230501038, 38, 3, 58, 98, 35], [230501039, 58, 80, 57, 100, 79], [230501040, 82, 1, 65, 89, 28], [230501041, 63, 72, 64, 65, 30], [230501042, 77, 68, 75, 50, 48], [230501043, 61, 69, 32, 84, 21], [230501044, 99, 62, 55, 23, 61], [230501045, 60, 60, 63, 64, 73], [230501046, 17, 72, 41, 46, 98], [230501047, 79, 6, 6, 11, 38], [230501048, 36, 91, 71, 39, 63], [230501049, 94, 3, 75, 32, 59], [230501050, 26, 50, 60, 77, 31], [230501051, 81, 55, 100, 7, 4], [230501052, 98, 48, 37, 40, 24], [230501053, 93, 33, 41, 26, 68], [230501054, 89, 94, 42, 98, 80], [230501055, 47, 66, 79, 50, 88], [230501056, 69, 36, 33, 14, 44], [230501057, 64, 4, 74, 73, 15], [230501058, 19, 83, 86, 82, 42], [230501059, 28, 6, 96, 3, 60], [230501060, 22, 41, 83, 6, 12], [230501061, 94, 59, 35, 30, 40], [230501062, 24, 100, 81, 95, 97], [230501063, 68, 

In [ ]:
test_fonksiyonlar()